In [13]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression

# Charger les données d'entraînement et de test
train_data = pd.read_csv('train(competion).csv')
test_data = pd.read_csv('test(competition).csv')

# Séparer les caractéristiques (features) et la cible dans les données d'entraînement
X_train, y_train = train_data.drop(columns=['Exited']), train_data['Exited']

# Identifier les colonnes catégorielles
categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()

# Créer des transformateurs pour l'encodage des variables catégorielles et la mise à l'échelle des caractéristiques numériques
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numerical_transformer = StandardScaler()

# Combiner les transformateurs
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_columns),
        ('num', numerical_transformer, X_train.select_dtypes(include=['number']).columns.tolist())
    ])

# Initialiser le modèle de régression logistique avec une augmentation du nombre d'itérations
model = LogisticRegression(max_iter=1000)

# Créer un pipeline incluant le prétraitement et le modèle
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('classifier', model)])

# Entraîner le modèle
pipeline.fit(X_train, y_train)

# Prétraitement des données de test (si nécessaire)
X_test = test_data

# Prédire la probabilité d'abandon ('Exited') pour les données de test
test_probabilities = pipeline.predict_proba(X_test)[:, 1]

# Créer un DataFrame pour les probabilités prédites
submission_df = pd.DataFrame({'id': test_data['id'], 'Exited': test_probabilities})

# Enregistrer les prédictions au format CSV sous le nom de prediction.csv
submission_df.to_csv('prediction.csv', index=False)
